# CNN with Keras by bgo - First Try

[Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python)의 가이드라인 대로 케라스를 사용하여 CNN 분류기를 학습시켜볼 것이다.

우리의 전력은 학습 데이터 12000개 중 20%를 검증 데이터로 사용하여 분류기를 최적화시키는 것이다. 테스트 데이터는 그대로 놔두고 정말 마지막에 모델의 정확도를 평가할 때 사용한다.

## 1. Generate dataset

In [18]:
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [19]:
# 데이터 로드
data_train = pd.read_csv('../data/fashion-mnist_train.csv')
data_test = pd.read_csv('../data/fashion-mnist_test.csv')

In [20]:
# 입력 이미지 크기
img_rows, img_cols = 28, 28    
input_shape = (img_rows, img_cols, 1)

# numpy 배열로 변환
X = np.array(data_train.iloc[:, 1:])
y = to_categorical(np.array(data_train.iloc[:, 0]))

In [21]:
# 학습/검증 데이터셋으로 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=13)

In [22]:
# 테스트 데이터도 numpy배열로 변환
X_test = np.array(data_test.iloc[:, 1:])
y_test = to_categorical(np.array(data_test.iloc[:, 0]))

In [23]:
# 1차원 데이터를 2차원 이미지 데이터로 변환
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)

# 0-1 값으로 정규화
X_train = X_train.astype('float32'); X_train /= 255;
X_test = X_test.astype('float32'); X_test /= 255;
X_val = X_val.astype('float32'); X_val /= 255;

## 2. Modeling

In [24]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization

In [25]:
batch_size = 256    # 배치 크기
num_classes = 10    # 분류할 클래스 개수
epochs = 50         # epoch

In [26]:
model = Sequential()

###################### 모델 설계 ######################
# Conv Layer 1
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',
                 kernel_initializer='he_normal', input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

# Conv Layer 2
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Conv Layer 3
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Dropout(0.4))

# 데이터 평탄화
model.add(Flatten())

# Dense Layer 1
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))

# Dense Layer 2
model.add(Dense(num_classes, activation='softmax'))
#########################################################

# 모델 컴파일
# 1. 최적화 알고리즘 : Adam
# 2. 비용 함수 : 카테고리형 Cross Entropy
# 3. 평가 지표 : 정확도
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 3, 128)         73856     
__________

## 3. Training

In [28]:
# 모델 학습
history = model.fit(X_train, y_train,
                   batch_size=batch_size, epochs=epochs,
                   verbose=1, validation_data=(X_val, y_val))

# 모델 성능 평가
score = model.evaluate(X_test, y_test, verbose=0)

Train on 48000 samples, validate on 12000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
48000/48000 [==============================] - 55s 1ms/sample - loss: 0.8314 - acc: 0.6919 - val_loss: 0.5245 - val_acc: 0.8026
Epoch 2/50
48000/48000 [==============================] - 56s 1ms/sample - loss: 0.5229 - acc: 0.8057 - val_loss: 0.4197 - val_acc: 0.8459
Epoch 3/50
16896/48000 [=========>....................] - ETA: 38s - loss: 0.4599 - acc: 0.8318

KeyboardInterrupt: 

In [ ]:
print('test loss :', score[0])
print('test accuracy :', score[1])

## 4. Results
### 4.1. Plotting accuracy and los

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline